<a href="https://colab.research.google.com/github/sugangnb/ai-research/blob/main/colab_notebooks/Kicked_dataset_catboost_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!rm -rf kicked_dataset/
!git clone https://github.com/calibertytz/kicked_dataset.git

Cloning into 'kicked_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 25 (delta 0), reused 0 (delta 0), pack-reused 22
Unpacking objects: 100% (25/25), done.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

df_train = pd.read_csv('kicked_dataset/df_train_final.csv')
df_test = pd.read_csv('kicked_dataset/df_test_final.csv')

train_X = df_train.drop(columns=['label']).values
train_Y = df_train['label'].values

X_test = df_test.drop(columns=['label']).values
Y_test = df_test['label'].values

test_X, val_X, test_Y, val_Y = train_test_split(X_test, Y_test, test_size=0.5, random_state=1) # x_val for updating alpha

In [ ]:
!pip install git+https://github.com/rwbfd/OpenCompetitionV2.git

  Cloning https://github.com/rwbfd/OpenCompetitionV2.git to /tmp/pip-req-build-3t340ow9
  Running command git clone -q https://github.com/rwbfd/OpenCompetitionV2.git /tmp/pip-req-build-3t340ow9
  Running command git submodule update --init --recursive -q
  Created wheel for open-competition: filename=open_competition-0.1-cp36-cp36m-linux_x86_64.whl size=3551758 sha256=291cb98b8a9986e0ac5bc73b0ba9c121c00aee0a4108f8e3e553bb5002c795a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-odq13xg2/wheels/7c/00/e9/62400be627e90fcd07373d244b8ede7e61c4e8640b5c4c5311
Successfully built open-competition


In [ ]:
from open_competition.tabular.model_fitter import CATFitter,CATOpt
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)

params = CATOpt().get_common_params()
params["thread_count"]=8
params["num_round"] = 1000
params["learning_rate"] = 1e-6
params['depth'] = 3
params['bootstrap_type'] = "MVS"
params['leaf_estimation_method'] = "Gradient"
params["max_bin"] = 100
params["early_stopping_rounds"]=1000
print(params)
train_pred, test_pred, acc_result = CATFitter().train_k_fold(k_fold=kf,train_data=df_train,test_data=df_test,params = params)
acc_result = [1-acc for acc in acc_result]
print(acc_result)
print(sum(acc_result)/5)

{'thread_count': 8, 'num_round': 1000, 'learning_rate': 1e-06, 'objective': 'CrossEntropy', 'eval_metric': 'Accuracy', 'depth': 2, 'bootstrap_type': 'MVS', 'leaf_estimation_method': 'Gradient', 'max_bin': 100, 'early_stopping_rounds': 1000}
The minimum is attained in round 4
The minimum is attained in round 2
The minimum is attained in round 13
The minimum is attained in round 21
The minimum is attained in round 2
[0.884, 0.8885, 0.8756, 0.8846, 0.8692]
0.88038
